In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from glob import glob
import cv2
import PIL
from PIL import Image
import json
from math import ceil
import pandas as pd

In [2]:
%matplotlib inline

In [11]:
train_path = '../dataset/train'
img_extension= '.jpg'

In [12]:
img_filenames = glob(train_path + '/**/*.jpg')

In [18]:
def getHeightWidth(filename):
    w, h = cv2.imread(filename, 0).shape
    return w, h

In [20]:
height_width_list = map(getHeightWidth, img_filenames)

In [23]:
width_list, height_list = zip(*height_width_list)

In [26]:
avg_width = np.mean(width_list)
avg_height = np.mean(height_list)

std_width = np.std(width_list)
std_height = np.std(height_list)

In [27]:
print(avg_width, avg_height, std_width, std_height)

205.814415691 166.597024387 49.0798965192 47.824756793


## Avg Width : 205.81
## Avg Height : 166.60
## Std Width : 49.08
## Std Height : 47.82

# Covers 95.4% of total images (stats)

In [29]:
upper_width_bound = int(avg_width + 2 * std_width)
lower_width_bound = int(avg_width - 2 * std_width)

upper_height_bound = int(avg_height + 2 * std_height)
lower_height_bound = int(avg_height - 2 * std_height)

In [30]:
def isProperWidth(w):
    return w >= lower_width_bound and w <= upper_width_bound 

def isProperHeight(h):
    return h >= lower_height_bound and h <= upper_height_bound

def isProperSize(filename):
    w, h = getHeightWidth(filename)
    return isProperWidth(w) and isProperHeight(h)

In [31]:
best_train_set = filter(isProperSize, img_filenames)

In [33]:
best_height_width_list = map(getHeightWidth, best_train_set)
best_width_list, best_height_list = zip(*best_height_width_list)

In [40]:
best_avg_width = ceil(np.mean(best_width_list))
best_avg_height = ceil(np.mean(best_height_list))

In [41]:
print(best_avg_width, best_avg_height)

209 162


In [49]:
def getLabel(filename):
    label = filename.split('/')[3]
    return int(label)

In [57]:
best_train_set = filter(isProperSize, img_filenames)
df = pd.DataFrame({'path' : list(best_train_set)})

In [59]:
df.tail()

,path
58262,../dataset/train/2/n03254046_1156_0.jpg
58263,../dataset/train/2/n03630383_2162_1.jpg
58264,../dataset/train/2/n03630383_3376_0.jpg
58265,../dataset/train/2/n03398228_6668_0.jpg
58266,../dataset/train/2/n04122492_3637_0.jpg


In [1]:
df.to_csv('../dataset/best_dataset.csv')

NameError: name 'df' is not defined

In [2]:
import tensorflow as tf

In [3]:
def conv2d(x, W, b, strides=1):
	x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
	x = tf.nn.bias_add(x, b)
	return tf.nn.relu(x)

def maxpool2d(x, k=2):
	return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [5]:
n_input = [None, 162, 209, 3]
n_classes = 15
dropout = 0.75

x = tf.placeholder(tf.float32, n_input)
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [24]:
def conv_net_vgg7(x, weights, biases, dropout):
    
        x = tf.reshape(x, shape=[-1, 162, 209, 3])
        conv1 = conv2d(x, weights['wc1'], biases['bc1'])
        conv1 = maxpool2d(conv1, 2)

        conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
        conv2 = maxpool2d(conv2, 2)
        
        conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
        conv3 = maxpool2d(conv3, 2)
        
        conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
        conv4 = maxpool2d(conv4, 2)
        
        conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
        conv5 = maxpool2d(conv5, 2)
        
        conv6 = conv2d(conv5, weights['wc6'], biases['bc6'])
        conv6 = maxpool2d(conv6, 1)
        
        fc1 = tf.reshape(conv6, [-1, weights['wd1'].get_shape().as_list()[0]])
        fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
        fc1 = tf.nn.relu(fc1)
        
        fc1 = tf.nn.dropout(fc1, dropout)
        out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
        
        return out

In [17]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, 3, 64])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wc3': tf.Variable(tf.random_normal([3, 3, 128, 256])),
    'wc4': tf.Variable(tf.random_normal([3, 3, 256, 512])),
    'wc5': tf.Variable(tf.random_normal([3, 3, 512, 512])),
    'wc6': tf.Variable(tf.random_normal([3, 3, 512, 512])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([6*7*512, 4096])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([4096, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([128])),
    'bc3': tf.Variable(tf.random_normal([256])),
    'bc4': tf.Variable(tf.random_normal([512])),
    'bc5': tf.Variable(tf.random_normal([512])),
    'bc6': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([4096])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [25]:
conv_net_vgg7(x, weights, biases, dropout)

<tf.Tensor 'Add_1:0' shape=(?, 15) dtype=float32>